In [1]:
!pwd
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124\run_objective_2l


'c:\\users\\81908\\appdata\\local\\pypoetry\\cache\\virtualenvs\\tfgpu-ehdmne1y-py3.8\\scripts\\python.exe'

In [2]:
# tensorflow2.0 + kerasでGPUメモリの使用量を抑える方法(最小限だけ使うように設定)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
import sys
sys.path.append(r'C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124')
import moa_MLPs_funcs

Tensorflow version 2.3.1
n_classes: 206
DEBUG


In [4]:
import optuna


def objective(trial):
    model_type = "2l"
    params = {}
    
    params["activation"] = trial.suggest_categorical("activation", ["relu", "elu", "selu"])
    
    denses = []
    for i in range(2):
        dense = trial.suggest_int(f'dense{i}', 512, 1500)
        denses.append(dense)
    params["denses"] = denses
    
    drop_rates = []
    for i in range(3):
        drop_rate = trial.suggest_uniform(f'drop_rate{i}', 0.1, 0.9)
        drop_rates.append(drop_rate)
    params["drop_rates"] = drop_rates
    
    params["sync_period"] = trial.suggest_int("sync_period", 1, 100)
    
    print("-" * 100)
    print(f"params: {params}")

    oof_score, Y_pred = moa_MLPs_funcs.train_and_evaluate(model_type=model_type, params=params)

    return oof_score

In [5]:
%%time

n_trials = 200
#n_trials = 50
#n_trials = 3

study = optuna.create_study(
    study_name="study",
    storage=f"sqlite:///study.db",
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=1),
)
study.optimize(objective, n_trials=n_trials)
study.trials_dataframe().to_csv(f"objective_history.csv", index=False)
with open(f"objective_best_params.txt", mode="w") as f:
    f.write(str(study.best_params))
print(f"\nstudy.best_params:\n{study.best_params}")

[I 2020-11-25 20:59:24,636] A new study created in RDB with name: study


----------------------------------------------------------------------------------------------------
params: {'activation': 'elu', 'denses': [747, 1420], 'drop_rates': [0.846045887470927, 0.2024995583434854, 0.8992324122593158]}



c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Epoch 1/2
12/12 [==============================] - 3s 230ms/step - loss: 0.5311 - binary_crossentropy: 0.5307 - val_loss: 0.2088 - val_binary_crossentropy: 0.2076
Epoch 2/2
12/12 [==============================] - 2s 173ms/step - loss: 0.0940 - binary_crossentropy: 0.0916 - val_loss: 0.0304 - val_binary_crossentropy: 0.0266
Epoch 1/2
12/12 [==============================] - 3s 221ms/step - loss: 0.5367 - binary_crossentropy: 0.5363 - val_loss: 0.2338 - val_binary_crossentropy: 0.2327
Epoch 2/2
12/12 [==============================] - 2s 161ms/step - loss: 0.0968 - binary_crossentropy: 0.0945 - val_loss: 0.0321 - val_binary_crossentropy: 0.0282


c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


...

Epoch 1/2
12/12 [==============================] - 3s 210ms/step - loss: 0.6221 - binary_crossentropy: 0.6217 - val_loss: 0.5420 - val_binary_crossentropy: 0.5411
Epoch 2/2
12/12 [==============================] - 2s 172ms/step - loss: 0.1237 - binary_crossentropy: 0.1218 - val_loss: 0.0515 - val_binary_crossentropy: 0.0485
Epoch 1/2
12/12 [==============================] - 3s 212ms/step - loss: 0.6186 - binary_crossentropy: 0.6182 - val_loss: 1.3443 - val_binary_crossentropy: 1.3444
Epoch 2/2
12/12 [==============================] - 2s 162ms/step - loss: 0.1142 - binary_crossentropy: 0.1123 - val_loss: 0.0523 - val_binary_crossentropy: 0.0487


c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Epoch 1/2
12/12 [==============================] - 3s 213ms/step - loss: 0.6273 - binary_crossentropy: 0.6269 - val_loss: 0.5568 - val_binary_crossentropy: 0.5560
Epoch 2/2
12/12 [==============================] - 2s 175ms/step - loss: 0.1265 - binary_crossentropy: 0.1247 - val_loss: 0.0478 - val_binary_crossentropy: 0.0442
Epoch 1/2
12/12 [==============================] - 3s 215ms/step - loss: 0.4724 - binary_crossentropy: 0.4717 - val_loss: 0.3903 - val_binary_crossentropy: 0.3883
Epoch 2/2
12/12 [==============================] - 2s 161ms/step - loss: 0.0628 - binary_crossentropy: 0.0600 - val_loss: 0.0402 - val_binary_crossentropy: 0.0362


[I 2020-11-25 21:01:34,020] Trial 2 finished with value: 0.040800293037547467 and parameters: {'activation': 'selu', 'dense0': 788, 'dense1': 766, 'drop_rate0': 0.35061881354582014, 'drop_rate1': 0.5196385276582971, 'drop_rate2': 0.4547623150236454}. Best is trial 1 with value: 0.025946270619158614.


oof_score: 0.040800293037547467
----------------------------------------------------------------------------------------------------

study.best_params:
{'activation': 'relu', 'dense0': 656, 'dense1': 641, 'drop_rate0': 0.24900816910213674, 'drop_rate1': 0.3764485816344382, 'drop_rate2': 0.41741397938453595}
Wall time: 2min 9s
